In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from tqdm.auto import tqdm
from itertools import product
from torchsummary import summary
import numpy as np
import shutil
import os

min_loss_so_far = float('inf')
tbPath = 'dlPipeline/runs/'
modelPath = 'dlPipeline/models/'
shutil.rmtree(modelPath, ignore_errors=True)
shutil.rmtree(tbPath, ignore_errors=True)
os.makedirs(modelPath)
os.makedirs(tbPath)

In [2]:
class CustomBrainMNIST(Dataset):
    def __init__(self, eventArrays: np.array, targetsArray: np.array, extends: tuple):
        eventTensors = torch.Tensor(eventArrays)
        targets = torch.tensor(targetsArray)
        self.eventTensors = eventTensors[extends[0]: extends[1]]
        self.targets = targets[extends[0]: extends[1]]
    
    def __len__(self):
        return self.targets.shape[0]
    
    def __getitem__(self, index):
        eeg = self.eventTensors[index]
        eegWithInChannel = eeg.reshape(1,4,256)
        target = self.targets[index].item()
        return eegWithInChannel, target

In [3]:
tensorDir = 'processedData/channelSlectedBandPassed/npy/eventArrays.npy'
targetsDir = 'processedData/channelSlectedBandPassed/npy/targets.npy'
npData = np.load(tensorDir)
npTargets = np.load(targetsDir)
randomIndex = np.random.permutation(npTargets.shape[0])
npData = npData[randomIndex]
npTargets = npTargets[randomIndex]
npTargets.shape, npData.shape

((64470,), (64470, 4, 256))

In [4]:
trainExtend = (0, 44470)
validationExtend = (44470, 54470)
testExtend = (54470, 64470)
training_data = CustomBrainMNIST(eventArrays=npData, targetsArray=npTargets, extends=trainExtend)
validation_data = CustomBrainMNIST(eventArrays=npData, targetsArray=npTargets, extends=validationExtend)
testing_data = CustomBrainMNIST(eventArrays=npData, targetsArray=npTargets, extends=testExtend)
len(training_data), len(validation_data), len(testing_data)

(44470, 10000, 10000)

In [5]:
class CNN001(nn.Module):
    def __init__(self):
        super().__init__()
        self.temporalConvolution1 = nn.Sequential(
            nn.Conv2d(1,32,(1,32)),
            nn.ReLU()
        )
        self.spatialConvolution = nn.Sequential(
            nn.Conv2d(32,25,(4,1)),
            nn.ReLU()
        )
        self.maxpool = nn.MaxPool2d((1,5))
        self.temporalConvolution2 = nn.Sequential(
            nn.Conv2d(25,32,(1,8)),
            nn.ReLU()
        )
        self.flatten = nn.Flatten()
        self.fcl = nn.Sequential(
            nn.Linear(38*32,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    
    def forward(self, x):
        out = self.temporalConvolution1(x)
        out = self.spatialConvolution(out)
        out = self.maxpool(out)
        out = self.temporalConvolution2(out)
        out = self.flatten(out)
        logits = self.fcl(out)
        return logits

In [6]:
def model_trainer(training_args: dict):
    global min_loss_so_far
    batch_size = int(training_args.get('batch_size', 8))
    device = training_args.get('dev_id')
    lr=float(training_args.get('lr',0.0001))
    epochs = int(training_args.get('num_epochs',10))
    tbPath = training_args['tbPath']
    modelPath = training_args['modelPath']
    # print((batch_size, device, lr, epochs, tbPath, modelPath))
    model_name = '_epochs=' + str(epochs) + '_lr=' + str(lr) + '_batchSize=' + str(batch_size)
    tb = SummaryWriter(log_dir=tbPath + model_name)

    training_dataloader = DataLoader(training_data, shuffle=True, batch_size=batch_size)
    validation_dataloader = DataLoader(validation_data, batch_size=batch_size)

    model = CNN001()
    #print(model)
    loss_criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model = model.to(device)
    train_step = 0
    val_step = 0

    for epoch in tqdm(range(epochs), desc='Epochs'):
        ## training
        model.train()
        train_loss = 0
        num_train_batches = len(training_dataloader)
        for batch, (X, y) in enumerate(training_dataloader):
            train_step += 1
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            loss = loss_criterion(pred, y)
            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss = loss.item()
            train_loss += loss
            tb.add_scalar('training loss vs step', loss, train_step)
        train_loss /= num_train_batches

        #validating
        val_size = len(validation_dataloader.dataset)
        num_val_batches = len(validation_dataloader)
        val_loss, correct = 0, 0
        model.eval()
        with torch.no_grad():
            for X, y in validation_dataloader:
                val_step += 1
                X = X.to(device)
                y = y.to(device)
                pred = model(X)
                loss = loss_criterion(pred, y).item()
                val_loss += loss
                tb.add_scalar('validation loss vs step', loss, val_step)
                correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        
        val_loss /= num_val_batches
        correct /= val_size
        accuracy = 100*correct
        if val_loss < min_loss_so_far:
            min_loss_so_far = val_loss
            checkpoint = dict()
            checkpoint['sd'] = model.state_dict()
            checkpoint['h_params'] = model_name
            checkpoint['saved_epoch'] = epoch
            torch.save(checkpoint, modelPath + 'bestModelSoFar.pt')
        tb.add_scalar('average training loss vs epoch', train_loss, epoch)
        tb.add_scalar('average validation loss vs epoch', val_loss, epoch)
        tb.add_scalar('validation accuracy vs epoch', accuracy, epoch)
    
    del model
    del training_dataloader
    del validation_dataloader    

In [7]:
# tempModel = CNN001().to(torch.device("cuda:1"))
# summary(tempModel, (1,4,256))
# del tempModel

In [7]:
parameters = dict(
    lr = [0.001, 0.01],
    batch_size = [100],
    epochs = [50]
)

param_values = [v for v in parameters.values()]
## performing hyper paramete tuning
device = torch.device("cuda:1") if torch.cuda.is_available() else torch.device("cpu")
print(device)
for run_id, (lr,batch_size, epochs) in enumerate(product(*param_values)):
    print("Run ID: {} | H Params==> lr:{}, batch_size:{}, epochs:{}".format(run_id, lr, batch_size, epochs))
    training_args = dict()
    training_args['num_epochs'] = epochs
    training_args['lr'] = lr
    training_args['batch_size'] = batch_size
    training_args['dev_id'] = device
    training_args['tbPath'] = tbPath
    training_args['modelPath'] = modelPath
    model_trainer(training_args)

cuda:1
Run ID: 0 | H Params==> lr:0.001, batch_size:100, epochs:50


2023-04-29 19:34:45.685333: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-29 19:34:46.811508: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-29 19:34:50.370074: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/sharmaanupam/.conda/envs/anp_103/lib64:/home/sharmaanupam/.conda/envs/anp_103/li

Epochs:   0%|          | 0/50 [00:00<?, ?it/s]

Run ID: 1 | H Params==> lr:0.01, batch_size:100, epochs:50


Epochs:   0%|          | 0/50 [00:00<?, ?it/s]

In [8]:
test_loader = DataLoader(testing_data)
## perfroming test
checkpoint = torch.load(modelPath + 'bestModelSoFar.pt')
model = CNN001()
model.load_state_dict(checkpoint['sd'])
print("Model hyper params:{}".format(checkpoint['h_params']))
print("Validation loss when the model was saved:{}".format(min_loss_so_far))
print("Epoch when the model was saved:{}".format(checkpoint['saved_epoch']))
loss_criteria = nn.CrossEntropyLoss()
test_size = len(test_loader.dataset)
num_test_batches = len(test_loader)
test_loss, correct = 0, 0
model.to(device)
model.eval()
with torch.no_grad():
    for X, y in test_loader:
        X = X.to(device)
        y = y.to(device)
        pred = model(X)
        loss = loss_criteria(pred, y).item()
        test_loss += loss
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
test_loss /= num_test_batches
correct /= test_size
accuracy = 100*correct
print(f"Test Error: \n Accuracy: {(accuracy):>0.1f}%, Avg loss: {test_loss:>8f} \n")
del model

Model hyper params:_epochs=50_lr=0.01_batchSize=100
Validation loss when the model was saved:2.302114398479462
Epoch when the model was saved:19
Test Error: 
 Accuracy: 9.9%, Avg loss: 2.302786 



: 